# Experiments with temporal sorted data split 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pickle as pickle
import pandas as pd
import numpy as np

import copy

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset

from src.trainer import CaseDataSet
from src.model import DLModels
from src.trainer import Trainer

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import importlib.util
torch_device = "cpu"
device_package = torch.cpu
if importlib.util.find_spec("torch.backends.mps") is not None:
    if torch.backends.mps.is_available():
        torch_device = torch.device("mps")
        device_package = torch.mps
if torch.cuda.is_available():
    torch_device = torch.device("cuda")
    device_package = torch.cuda
    
torch_device

device(type='cuda')

In [8]:
def train_source_model(datasets_dict, model_hyper_para, trainer_hyper_para,
                       split, embedding, print_iter=False, file_suffix=""):
    torch_device = "cpu"
    device_package = torch.cpu
    if importlib.util.find_spec("torch.backends.mps") is not None:
        if torch.backends.mps.is_available():
            torch_device = torch.device("mps")
            device_package = torch.mps
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        device_package = torch.cuda
        
    current_model_para = model_hyper_para[embedding]
    # Instantiate the model
    model = DLModels.SimpleLSTM(current_model_para["input_size"],
                                current_model_para["hidden_size"],
                                current_model_para["num_layers"],
                                1).to(torch_device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=trainer_hyper_para["learning_rate"])
    
    # Load data sets
    source_data_sets = datasets_dict[split][embedding]["source"]
    

    
    print(torch_device)
    print("training model with target data")
    model_source, train_loss_source, val_loss_srouce = Trainer.train_model(model, optimizer,
                                                                           None, None,
                                                                           source_data_sets["train"],
                                                                           source_data_sets["val"],
                                                                           trainer_hyper_para["batch_size"],
                                                                           torch_device,
                                                                           device_package,
                                                                           Trainer.prefix_weighted_loss,
                                                                           trainer_hyper_para["max_epoch"],
                                                                           trainer_hyper_para["max_ob_iter"],
                                                                           trainer_hyper_para["score_margin"],
                                                                           print_iter=print_iter)
    
    
    model_name = "LSTM_S_h" + str(current_model_para["hidden_size"]) + "_l" + str(current_model_para["num_layers"]) + "_" + embedding
    torch.save(model_source, "../../Model/" + split + "/LSTM/"+ model_name + "_" +  file_suffix + ".LSTM")
    
    training_stat = pd.DataFrame(columns=["TrainingLoss", "ValidationLoss"],
                                 data=np.hstack([train_loss_source.reshape((-1, 1)),
                                                 val_loss_srouce.reshape((-1, 1))]))
    training_stat.to_pickle("../../Model/" + split + "/LSTM/"+ model_name + "_" + file_suffix + "_stat.pkl")
    
    
    print("Finished training model with target data")
    
def train_target_model(datasets_dict, model_hyper_para, trainer_hyper_para,
                       split, embedding, print_iter=False, file_suffix=""):
    torch_device = "cpu"
    device_package = torch.cpu
    if importlib.util.find_spec("torch.backends.mps") is not None:
        if torch.backends.mps.is_available():
            torch_device = torch.device("mps")
            device_package = torch.mps
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        device_package = torch.cuda
        
    current_model_para = model_hyper_para[embedding]
    # Instantiate the model
    model = DLModels.SimpleLSTM(current_model_para["input_size"],
                                current_model_para["hidden_size"],
                                current_model_para["num_layers"],
                                1).to(torch_device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=trainer_hyper_para["learning_rate"])
    
    # Load data sets
    data_sets = datasets_dict[split][embedding]["target"]
    

    
    print(torch_device)
    print("training model with target data")
    model_source, train_loss_source, val_loss_srouce = Trainer.train_model(model, optimizer,
                                                                           None, None,
                                                                           data_sets["train"],
                                                                           data_sets["val"],
                                                                           4000,
                                                                           torch_device,
                                                                           device_package,
                                                                           Trainer.prefix_weighted_loss,
                                                                           trainer_hyper_para["max_epoch"],
                                                                           trainer_hyper_para["max_ob_iter"],
                                                                           trainer_hyper_para["score_margin"],
                                                                           print_iter=print_iter)
    
    
    model_name = "LSTM_T_h" + str(current_model_para["hidden_size"]) + "_l" + str(current_model_para["num_layers"]) + "_" + embedding
    torch.save(model_source, "../../Model/" + split + "/LSTM/"+ model_name + "_" +  file_suffix + ".LSTM")
    
    training_stat = pd.DataFrame(columns=["TrainingLoss", "ValidationLoss"],
                                 data=np.hstack([train_loss_source.reshape((-1, 1)),
                                                 val_loss_srouce.reshape((-1, 1))]))
    training_stat.to_pickle("../../Model/" + split + "/LSTM/"+ model_name + "_" + file_suffix + "_stat.pkl")
    
    
    print("Finished training model with target data")

In [ ]:
training_stat = pd.read_pickle("../../Model/641620split/LSTM/LSTM_S_h512_l1_st_stat.pkl")

In [6]:
trainer_hyper_para = {"max_epoch": 100,
                      "max_ob_iter": 20,
                      "score_margin": 1,
                      "num_class": 1,
                      "num_layers": 1,
                      "learning_rate": 1e-3,
                      "batch_size": 50,
                      "training_loss_func": "CCM",
                      "eval_loss_func": "CCM"}

model_hyper_para = {"w2v": {"input_size": 51,
                            "hidden_size": 128,
                            "num_layers": 1},
                    "st": {"input_size": 385,
                           "hidden_size": 512,
                           "num_layers": 1},
                    "onehot": {"input_size": 48,
                               "hidden_size": 128,
                               "num_layers": 1}}


split_pattern = ["811split", "641620split"]
data_set_type = ["source", "target"]
data_set_cat = ["train", "val", "test"]
embedding_type = ["w2v", "st", "onehot"]
dataset_list_index = [split_pattern, embedding_type, data_set_type, data_set_cat]

earliness_requirement = True
folder_path = "../../Data/Training/"

In [7]:
data_sets_list = {}
for s_pattern in split_pattern:
    data_split_list = {}
    for embedding in embedding_type:
        data_embedding_list = {}
        for d_type in data_set_type:
            data_type_list = {}
            for d_cat in data_set_cat:       
                data_version = "_" + d_cat + "_sorted"
                embedding_version = "_" + embedding
                case_data_set = CaseDataSet.CaseDataset(split_pattern=s_pattern,
                                                        input_data=d_type,
                                                        data_version=data_version,
                                                        embedding_version=embedding_version,
                                                        earliness_requirement=earliness_requirement)
                
                data_type_list[d_cat] = case_data_set
            data_embedding_list[d_type] = data_type_list
        data_split_list[embedding] = data_embedding_list
    data_sets_list[s_pattern] = data_split_list

In [10]:
for i in range(10):
    train_source_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="w2v", print_iter=False, file_suffix=str(i))
    train_source_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="st", print_iter=False, file_suffix=str(i))
    train_source_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="onehot", print_iter=False, file_suffix=str(i))
    train_target_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="w2v", print_iter=False, file_suffix=str(i))
    train_target_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="onehot", print_iter=False, file_suffix=str(i))
    train_target_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                       split="641620split", embedding="st", print_iter=False, file_suffix=str(i))

cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model with target data
cuda
training model with target data
Finished training model wit

In [9]:
train_target_model(data_sets_list, model_hyper_para, trainer_hyper_para,
                   split="641620split", embedding="onehot", print_iter=True, file_suffix="b2")

cuda
training model with target data
Finished training iteration:  0  with val loss:  2037.6947858247484  train loss:  4621.937561457762
Finished training iteration:  1  with val loss:  2117.900303688917  train loss:  4698.674602248174
Finished training iteration:  2  with val loss:  2219.5506409357904  train loss:  4802.460654858565
Finished training iteration:  3  with val loss:  2067.208663561274  train loss:  4934.881693740298


KeyboardInterrupt: 